In [9]:
# Use autoreload to automatically reload modules
%load_ext autoreload
%autoreload 2
import rootutils
root_path = rootutils.find_root()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import rioxarray as rxr
import pandas as pd
import cubo
from burned_embedder.data import load_s2, load_s1
import ee
ee.Authenticate()
ee.Initialize()

In [11]:
def load_dynamic_world(lat, lon, start_date="2025-01-01", end_date="2025-12-31", edge_size=100):
    """Load and clean Dynamic WWorld Data"""

    da_dyn_world = cubo.create(
        lat=lat, lon=lon,
        collection="GOOGLE/DYNAMICWORLD/V1",
        bands=["trees"],
        start_date=start_date, end_date=end_date,
        edge_size=edge_size, resolution=10,
        gee=True

    )

    # Clean duplicates
    da_dyn_world = da_dyn_world.drop_duplicates(dim='time', keep='first')

    print(f"S1 data shape: {da_dyn_world.shape}")

    return da_dyn_world

In [23]:
dyn = load_dynamic_world(lat=33.0, lon=-117.0, edge_size=128)
print(dyn.epsg)

S1 data shape: (167, 1, 128, 128)
32611


In [27]:
s1_da = load_s1(lat=33.0, lon=-117.0, edge_size=128)
print(s1_da.epsg)

Loading Sentinel-1 data...
S1 data shape: (199, 2, 128, 128)
<xarray.DataArray 'epsg' ()> Size: 8B
array(32611)
Coordinates: (12/20)
    sar:frequency_band           <U1 4B 'C'
    sar:resolution_azimuth       int64 8B 22
    sar:looks_equivalent_number  float64 8B 4.4
    sar:looks_range              int64 8B 5
    sar:pixel_spacing_azimuth    int64 8B 10
    sar:polarizations            object 8B {'VV', 'VH'}
    ...                           ...
    sar:resolution_range         int64 8B 20
    constellation                <U10 40B 'Sentinel-1'
    sar:center_frequency         float64 8B 5.405
    s1:processing_level          <U1 4B '1'
    raster:bands                 object 8B {'nodata': -32768, 'data_type': 'f...
    epsg                         int64 8B 32611


In [24]:
s2_da = load_s2(lat=33.0, lon=-117.0, edge_size=128, bands=["B0", "B03", "B02"], max_cloud_cover=100)
print(s2_da.epsg)


Loading Sentinel-2 data...
Applying baseline harmonization to bands: ['B02', 'B03']
S2 data shape: (241, 2, 128, 128)
<xarray.DataArray 'epsg' ()> Size: 8B
array(32611)
Coordinates:
    s2:product_type                          <U7 28B 'S2MSI2A'
    s2:saturated_defective_pixel_percentage  float64 8B 0.0
    instruments                              <U3 12B 'msi'
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    proj:code                                <U10 40B 'EPSG:32611'
    constellation                            <U10 40B 'Sentinel 2'
    sat:orbit_state                          <U10 40B 'descending'
    gsd                                      float64 8B 10.0
    proj:shape                               object 8B {10980}
    epsg                                     int64 8B 32611


In [29]:


comp = rxr.open_rasterio(root_path / "data/interim/radd/00N_050W_radd_alerts/sample_03_components.tif")

In [33]:
comp

<xarray.DataArray (band: 1, y: 5000, x: 5000)> Size: 50MB
[25000000 values with dtype=uint16]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40kB -45.15 -45.15 -45.15 ... -44.65 -44.65 -44.65
  * y            (y) float64 40kB -6.063 -6.063 -6.063 ... -6.562 -6.562 -6.562
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0